In [ ]:
%load_ext autoreload
%autoreload 2

# Paramètres de la pipeline

In [ ]:
code_departement = "093"
logs_level = "WARNING"

# Imports & setup

In [ ]:
import geopandas as gpd
import pandas as pd

from potentiel_solaire.attach_buildings_to_schools import attach_buildings_to_schools
from potentiel_solaire.constants import DEFAULT_CRS, RESULTS_FOLDER
from potentiel_solaire.sources.bd_topo import extract_bd_topo, get_topo_zones_of_interest, \
    get_topo_buildings_of_interest
from potentiel_solaire.sources.schools_establishments import extract_schools_establishments, \
    get_schools_establishments_of_interest
from potentiel_solaire.sources.protected_buildings import extract_protected_buildings, get_areas_with_protected_buildings
from potentiel_solaire.features.solar_potential import calculate_solar_potential
from potentiel_solaire.aggregate import aggregate_solar_potential_by_etablishment
from potentiel_solaire.duckdb_manager import save_solar_potential_by_school, \
    save_solar_potential_by_commune, \
    save_solar_potential_by_department
from potentiel_solaire.geojson_files_creator import export_to_geojson
from potentiel_solaire.logger import get_logger

logger = get_logger()
logger.setLevel(logs_level)
pd.options.display.max_columns = None

# Extraction des données sources

### Etablissements scolaires

In [ ]:
schools_establishments_path = extract_schools_establishments()
print(f"Annuaire des établissements scolaires extrait ici: {schools_establishments_path}")

### BD TOPO

In [ ]:
bd_topo_path = extract_bd_topo(code_departement=code_departement)
print(f"BD TOPO extraite ici: {bd_topo_path}")

### BD Protected Buildings

In [ ]:
bd_protected_buildings_path = extract_protected_buildings()
print(f"BD des bâtiments protégés extraite ici: {bd_protected_buildings_path}")

# Filtre des données sur le périmètre du calcul

### Etablissements scolaires


In [ ]:
schools_establishments = get_schools_establishments_of_interest(
    schools_filepath=schools_establishments_path,
    code_departement=code_departement,
    types_etablissements=['Ecole', 'Lycée', 'Collège'],
    statut_public_prive="Public",
    etat="OUVERT",
    crs=DEFAULT_CRS
)
nb_schools = schools_establishments.shape[0]
print(f"Nb d'établissements scolaires: {nb_schools}")

### Zone d'intérêt géographique

In [ ]:
codes_commune = schools_establishments["code_commune"].unique()
communes = gpd.read_file(bd_topo_path, layer="commune").to_crs(DEFAULT_CRS)
communes = communes[communes.code_insee.isin(codes_commune)]
geom_of_interest = communes.dissolve()[["geometry"]]

### Zones d'éducations

In [ ]:
educational_zones = get_topo_zones_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    categories=["Science et enseignement"],
    natures=['Collège', 'Lycée', 'Enseignement primaire'],
    crs=DEFAULT_CRS
)
nb_educational_zones = educational_zones.shape[0]
print("Nb de zones d'éducations: ", nb_educational_zones)

### Bâtiments

In [ ]:
buildings = get_topo_buildings_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)
nb_buildings = buildings.shape[0]
print("Nb de batiments: ", nb_buildings)

### Zones avec des bâtiments protégés

In [ ]:
areas_with_protected_buildings = get_areas_with_protected_buildings(
    bd_protected_buildings_path=bd_protected_buildings_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)

# Détermination des bâtiments scolaires

In [ ]:
schools_buildings = attach_buildings_to_schools(
    schools_establishments=schools_establishments,
    educational_zones=educational_zones,
    buildings=buildings
)
nb_schools_buildings = schools_buildings.shape[0]
print("Nb de batiments scolaires: ", nb_schools_buildings)

# Calcul des attributs utiles pour le potentiel solaire

In [ ]:
# TODO: v0 seulement à ce stade
solar_potential_of_schools_buildings = calculate_solar_potential(
    schools_buildings=schools_buildings,
    areas_with_protected_buildings=areas_with_protected_buildings,
    geom_of_interest=geom_of_interest
)

# Dump des donnees pour analyses

In [ ]:
layers = ["schools_establishments", "educational_zones", "schools_buildings", "solar_potential_of_schools_buildings"]
gdfs = [schools_establishments, educational_zones, schools_buildings, solar_potential_of_schools_buildings]

for layer, gdf in zip(layers, gdfs):
    output_gpkg = RESULTS_FOLDER / f"D{code_departement}_pipeline_results.gpkg"
    gdf.to_file(output_gpkg, layer=layer, driver="GPKG")

# Checks sur la qualité des données & calculs

In [ ]:
nb_schools_with_buildings = len(schools_buildings.identifiant_de_l_etablissement.unique())
print("Nb d'établissements scolaires avec des batiments: {} ({}%)".format(
    nb_schools_with_buildings,
    round(100 * nb_schools_with_buildings / nb_schools)
))

In [ ]:
nb_buildings_protected = solar_potential_of_schools_buildings[solar_potential_of_schools_buildings.protection].shape[0]
print(f"Nb de batiments protégés {nb_buildings_protected} ({100*nb_buildings_protected/nb_schools_buildings:.2f}%)")

### Evaluation du Potentiel solaire par Etablissement

In [ ]:
results_by_school = aggregate_solar_potential_by_etablishment(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings
)

results_by_school.head()

# Sauvegarde des calculs en Base de Données

#### Mise à jour des données des Etablissements du Département en Base de Données

In [ ]:
stats_schools= save_solar_potential_by_school(
    results_by_school,
    code_departement
)

assert stats_schools.query('surface_utile > 0').shape[0] > 0, "Problème lors de la mise à jour des établissements"

stats_schools.head(3)

#### Mise à jour des données des Communes du Département en Base de Données

In [ ]:
stats_cities = save_solar_potential_by_commune(
    code_departement
)

assert stats_cities.query('surface_utile > 0').shape[0] > 0, "Problème lors de la mise à jour des communes"

stats_cities.head(3)

#### Mise à jour des données du Département en Base de données

In [ ]:
stats_department = save_solar_potential_by_department(
    code_departement
)

assert stats_department['surface_utile'].shape[0] > 0, f"Problème lors de la mise à jour du département {code_departement}"

stats_department.head(3)

### Export sous forme de fichiers GeoJSON

In [ ]:
output_folder = RESULTS_FOLDER
output_folder.mkdir(exist_ok=True, parents=True)

export_to_geojson(stats_schools, output_folder / f"D{code_departement}_ecoles.geojson")
export_to_geojson(stats_cities, output_folder / f"D{code_departement}_communes.geojson")
export_to_geojson(stats_department, output_folder / f"D{code_departement}_departement.geojson")